In [256]:
import requests
from bs4 import BeautifulSoup
from itertools import chain
import pandas as pd 
from multiprocessing import Pool
import re



In [2]:
def getSoupFromURL(url, method='get', data={}):

  if method == 'get':
    res = requests.get(url)
  elif method == 'post':
    res = requests.post(url, data=data)
  else:
    return None

  if res.status_code == 200:
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup
  else:
    return None

In [65]:
soup = getSoupFromURL('https://www.leboncoin.fr/voitures/offres/ile_de_france/?q=renault%20zoe')

In [42]:
a = soup.find_all('a', class_ = 'list_item clearfix trackable')

In [86]:
def getLinkspage(urlpage):
    soup = getSoupFromURL(urlpage)
    car_links = []
    for article in soup.find_all('a', class_ = 'list_item clearfix trackable'):
        car_links.append(article['href'].replace('//',''))
    return car_links

In [ ]:
def getPagesLinks():
    soup = getSoupFromURL(urlpage)
    nextpage = soup.find_all('a',id= 'next')
    nextpage[0]['href']

In [157]:
def get_urls_all_pages_of_regions():  
    
    urls = []
    
    
    regions = ["provence_alpes_cote_d_azur", "ile_de_france","aquitaine"]

    for region in regions:
        url = 'https://www.leboncoin.fr/voitures/offres/'+ region +'/?o=1&q=renault%20zoe'
        urls.append(url)
        while_check =1
        while(while_check == 1):

            try:
                soup = getSoupFromURL(url)
                nextpage = soup.find_all('a',id= 'next')
                url = 'https:' + nextpage[0]['href']
                urls.append(url)

            except IndexError:
                while_check = 0
    return urls


In [275]:
def get_informations(url):
    soup = getSoupFromURL(url)
    
    #get title 
    title = soup.find("h1", class_ = "no-border").text.strip()

    #get infos
    a = soup.find_all('div', class_ = 'line')
    a[6].find(class_ = 'value').text
    a[4].find(class_ = 'property').text
    
    #prix
    price = int(a[1].find(class_ = 'value').text.strip()\
                .replace('\xa0€','')\
                .replace(' ',''))
   
    #annee
    annee = int(a[5].find(class_ = 'value').text.strip())
   
    #kilometrage
    kilometrage = int(a[6].find(class_ = 'value').text\
                     .replace('KM','')\
                     .replace(' ',''))
    #telephone
    telephone = 'NA'
    
    #profesional
    pro = soup.find_all("span", class_ = "ispro")
    
    if len(pro) > 0:
        ispro = True
    else:
        ispro = False
        
    #version
    model = re.findall(r'(zen|intens|life)', str(title.lower()))
    if(len(model)>0):
        model = model[0]
    else:
        model = "NA"
    
    
    return title, price, annee, kilometrage, telephone, profesional, version 

    
    

In [274]:
if __name__ == '__main__':
    
    links = get_urls_all_pages_of_regions()
    car_links =[]
    
    
    for link in links:
        car_links.append(getLinkspage(link))
    car_links = list(chain(car_links))[0]
    
    car_links_df = pd.DataFrame(car_links, columns= ['car_links'])
    
    pool = Pool(processes=8)
    df_zoe = pool.map(extract_info_from_link, links)
    
        
    
    


ConnectionError: HTTPSConnectionPool(host='www.leboncoin.fr', port=443): Max retries exceeded with url: /voitures/offres/provence_alpes_cote_d_azur/?o=1&q=renault%20zoe (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x1142fe2e8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [232]:
https://www.lacentrale.fr/cote-voitures-renault-zoe--2016-.html